In [ ]:
############################################## SETTINGS ##############################################  
import os
from os import walk
from tqdm import tqdm
import pickle
import random 
import csv
from datetime import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
# Neuron
from   neuron           import h
import neuron as nrn
nrn.load_mechanisms('mechanisms/single')
# import custom functions
from   master_functions import *
#  Setup neuron
h.load_file('stdlib.hoc')
h.load_file('import3d.hoc')

############################################## SETTINGS ##############################################  
# original: original = True, method = 0 and physiological = False
# original = False        # inherited code
method = 0                # if 0 stimulate away from soma; if 1 then stimulate towards
physiological = True      # if False then uses original values else more physiological settings for phasic conductances 
# if model = 0 then original cell; if model = 1 then paper 
model = 1
cell_type='dspn'
specs = {'dspn': {
                    'N': 71,
                    'lib': 'Libraries/D1_71bestFit_updRheob.pkl',
                    'model': 2,
                    'morph': 'Morphologies/WT-dMSN_P270-20_1.02_SGA1-m24.swc',
                    },
         'ispn': {
                    'N': 34,
                    'lib': 'Libraries/D2_34bestFit_updRheob.pkl',
                    'model': 2,
                    'morph': 'Morphologies/WT-iMSN_P270-09_1.01_SGA2-m1.swc'}
        }


specs[cell_type]['model'] = model
morphology = specs[cell_type]['morph']
if model == 0:
    method = 0    
if model == 2:
    v_init = -84
else:
    v_init = -83.7155
return_currents = False
return_Ca = False
change_Ra = False
spine_per_length=1.711
tonic = False
gaba_locations = None # if None then default placement is the midpoint of the dendrite
figs_show = True

# nb currents in pA; Neuron default is nA
vary_hc = False
holding_current = 0


In [ ]:
sim = '320a'

In [ ]:
############################################## Figure 7 ############################################### 
########################################## Figure 7 B-D    ############################################
# glutamate submaximal 
# dend[15]
# #glut 15
# No phasic GABA
# tonic GABA
# no rectification
# uniform distribution
# long burn time as tonic GABA takes a while to achieve eqm; sim time 600 ms (>600s per sim)
# range of tonic GABA conductances

glut = True
if glut:
    if cell_type == 'dspn':
        num_gluts = 15
    elif cell_type == 'ispn':
        num_gluts = 12
else:
    num_gluts = 0

g_AMPA = 0.001 # original setting 0.001
AMPA = True
NMDA = True
ratio = 2.15 # NMDA to AMPA conductance ratio (not necessarily the current ratio)
glut_placement = 'distal'
if physiological:
    gaba_weight = 0.001
else:
    gaba_weight = 0.1 # tuned for non-distributed gaba (will be higher depolarization than ~5mv if running whole cell)

gaba_reversal = -60 # -83.7155 # 
# Phasic gaba
gaba = False

sim_time = 600
vary_gaba = False # if true timing of gaba is varied relative to glut; if false then vary glut relative to gaba

# if vary_gaba is True then gaba timing is given by timing  and glut timing is given by stim_time
# if vary_gaba is False then glut timing is given by timing and gaba timing is given by stim_time
stim_time = 350 # 150

spine_neck_diam = 0.1 # original setting 0.1

# 15 dendrites whose midpoint is furthest from the soma

dend_gaba =  []

num_gabas = 0
if not gaba:
    num_gabas = 0

# where to position glutamate
dend_glut = ['dend[15]']
glut_frequency = 1000 # every 1ms
gaba_frequency = None # every 1ms

# Tonic GABA
tonic = True
rectification = False
distributed = False # if want a funky distribution of tonic GABA ie concentrated on soma:

gbar_gaba_range = np.logspace(np.log10(1e-6), np.log10(3e-2), 100)
gaba_range = np.repeat(gaba, len(gbar_gaba_range)) # 

num_gabas_range = []
for x in gaba_range:
    num_gabas_range.append(x*num_gabas)
glut_range = np.repeat(glut, len(gbar_gaba_range))
num_gluts_range = []
for x in glut_range:
    num_gluts_range.append(x*num_gluts)
timing_range = np.repeat(stim_time, len(gbar_gaba_range))
black_trace = 0
gray_trace = None

impedance = False
# impedance is defined in frequency domain
freq = 10 

return_currents = True
save = True

In [ ]:
# Save figures
def save_all():
    import datetime 
    time = datetime.datetime.now()
    if os.name == 'nt':
        time = time.strftime("%Y-%m-%d_%H-%M-%S")
        
    path_cell = "{}".format(cell_type)
    if not os.path.exists(path_cell):
        os.mkdir(path_cell)    
    path1 = "{}/model{}".format(path_cell, model)
    if not os.path.exists(path1):
        os.mkdir(path1)
    if physiological: 
        path2 = "{}/physiological".format(path1)
    else:
        path2 = "{}/nonphysiological".format(path1)
    if not os.path.exists(path2):
        os.mkdir(path2)  
    path3 = "{}/images".format(path2)
    if not os.path.exists(path3):
        os.mkdir(path3)
    path4 = "{}/simulations".format(path2)
    if not os.path.exists(path4):
        os.mkdir(path4)  
    
    if sim in [307]:
        path5 = "{}/sim{}".format(path3, sim)
        if not os.path.exists(path5):
            os.mkdir(path5)    
        path6 = "{}/sim{}".format(path4, sim)
        if not os.path.exists(path6):
            os.mkdir(path6)  
            
        image_dir = "{}/sim{}".format(path5, iii)
        if not os.path.exists(image_dir):
            os.mkdir(image_dir)    
        sim_dir = "{}/sim{}".format(path6, iii)
        if not os.path.exists(sim_dir):
            os.mkdir(sim_dir)  
    else:
        image_dir = "{}/sim{}".format(path3, sim)
        if not os.path.exists(image_dir):
            os.mkdir(image_dir)    
        sim_dir = "{}/sim{}".format(path4, sim)
        if not os.path.exists(sim_dir):
            os.mkdir(sim_dir)  
        
    names = list(data_dict.keys())

    for name in names:
        with open('{}/{}.pickle'.format(sim_dir, name), 'wb') as handle:
            pickle.dump(data_dict[name], handle, protocol=pickle.HIGHEST_PROTOCOL)  
   

In [ ]:
def phasic_current_clamp(sim_time, glut_time, gaba_time, glut, num_gluts, dend_glut, gaba, num_gabas, gaba_weight, dend_gaba, gaba_locations, spine_neck_diam, return_currents, cell_type='dspn', plot=True, path=True):
    # Initialise cell properties
    cell, spines, dend_tree = cell_build(cell_type=cell_type, specs=specs, addSpines=True, spine_per_length=spine_per_length)

    if 'g_name' in globals():
        g_alter(cell=cell, spines=spines, g_name=g_name, g8=g8, specs=specs, cell_type=cell_type)
        print('g{} = {}'.format(g_name, g8))

    # nb currents in pA; Neuron default is nA
    hc = holding_current/1e3
    
    if tonic:
        if distributed:
            tonic_gaba(cell=cell, gaba_reversal=gaba_reversal, gbar_gaba=gbar_gaba, d3=gaba_params['d3'], a4=gaba_params['a4'], a5=gaba_params['a5'], a6=gaba_params['a6'], a7=gaba_params['a7'], rectification=rectification)                
        else:
            tonic_gaba(cell=cell, gaba_reversal=gaba_reversal, gbar_gaba=gbar_gaba, rectification=rectification)
 
    # Change all spine neck diameters
    spine_neck_diameter(cell=cell, spines=spines, diam=spine_neck_diam)

    dstep1 = int(1 / glut_frequency * 1e3)
    if gaba_frequency is not None:
        dstep2 = int(1 / gaba_frequency * 1e3)

    glut_secs_orig = [sec for target_dend in dend_glut for sec in cell.dendlist if sec.name() == target_dend] 
    glut_secs = [sec for target_dend in dend_glut for sec in cell.dendlist if sec.name() == target_dend] * num_gluts

    gaba_secs = [sec for target_dend in dend_gaba for sec in cell.dendlist if sec.name() == target_dend] * num_gabas

    # Phasic glut
    glut_onsets = list(range(glut_time, glut_time + num_gluts * dstep1, dstep1)) 
    # Phasic gaba
    if 'rel_gaba_onsets' in globals():
        gaba_onsets = [x + gaba_time for x in rel_gaba_onsets]
    else:
        if gaba_frequency is None:
            gaba_onsets = [gaba_time] * len(gaba_secs)
        else:
            gaba_onsets = list(range(gaba_time, gaba_time + num_gabas * dstep2, dstep2)) * len(gaba_secs)

    # Place glutamatergic synapses
    glut_synapses, glut_stimulator, glut_connection, ampa_currents, nmda_currents = glut_place(
        spines=spines,
        method=method,
        physiological=physiological,
        AMPA=AMPA,
        g_AMPA=g_AMPA,
        NMDA=NMDA,
        ratio=ratio,
        glut_time=glut_time,
        glut_secs=glut_secs,
        glut_onsets=glut_onsets,
        num_gluts=num_gluts,
        return_currents=return_currents,
        model=model
    )

    # Place gabaergic synapses
    gaba_synapses, gaba_stimulator, gaba_connection, gaba_currents, gaba_conductances = gaba_place(
        physiological=physiological,
        gaba_reversal=gaba_reversal,
        gaba_weight=gaba_weight,
        gaba_time=gaba_time,
        gaba_secs=gaba_secs,
        gaba_onsets=gaba_onsets,
        gaba_locations = gaba_locations,
        num_gabas=num_gabas,
        return_currents=return_currents
    )

    dend = glut_secs[0] if glut else glut_secs_orig[0] if gaba else None

    # Record vectors
    if path:
        pathlist = path_finder(cell=cell, dend_tree=dend_tree, dend=dend)
        v = record_v(cell=cell, seclist=pathlist, loc=0.4)
        ca = record_cai(cell, seclist=pathlist, loc=0.4, return_Ca=return_Ca)
    else:
        v = record_all_v(cell, loc=0.4)
        ca = record_all_cai(cell, seclist=pathlist, loc=0.4, return_Ca=return_Ca)
    t = h.Vector().record(h._ref_t)
    i_mechs_dend = record_i_mechs(cell=cell, dend=dend, loc=0.4, return_currents=return_currents)

    # add holding current
    iclamp1 = h.IClamp(cell.soma(0.5))
    iclamp1.dur = sim_time
    iclamp1.amp = hc # nA

    # Initialize cell starting voltage
    h.finitialize(v_init)

    # Run simulation
    dt = 0.025
    h.dt = dt

    if impedance:
        # Set up vectors
        imp, zvec1, zvec2 = record_impedance(dend=dend, loc=0.4)
        while h.t < sim_time:
            imp.compute(freq, 1)
            zvec1.append(imp.input(0.4, sec=dend))
            zvec2.append(imp.input(0.4, sec=cell.soma))
            h.fadvance()
    else:
        while h.t < sim_time:
            h.fadvance()

    # Get sections to plot
    if path:
        seclist = pathlist
    else:
        seclist = dend2plot(cell)

    tree_plots = False
    if tree_plots:
        fig_path = plot1(cell=cell, dend=dend, t=t, v=v, seclist=seclist, sparse=False, protocol='')
        fig_path.show()
        if return_Ca:
            fig_path_Ca = plot1_Ca(cell=cell, dend=dend, t=t, Ca=ca, seclist=seclist, sparse=False, protocol='')
            fig_path_Ca.show()

    if impedance:
        if return_Ca:
            return v['soma[0]'], v[dend.name()], v, ca['soma[0]'], ca[dend.name()], ca, t, i_mechs_dend, ampa_currents, nmda_currents, gaba_currents, gaba_conductances, zvec1, zvec2, dt
        else:
            return v['soma[0]'], v[dend.name()], v, t, i_mechs_dend, ampa_currents, nmda_currents, gaba_currents, gaba_conductances, zvec1, zvec2, dt
    else:
        if return_Ca:
            return v['soma[0]'], v[dend.name()], v, ca['soma[0]'], ca[dend.name()], ca, t, i_mechs_dend, ampa_currents, nmda_currents, gaba_currents, gaba_conductances, dt
        else:
            return v['soma[0]'], v[dend.name()], v, t, i_mechs_dend, ampa_currents, nmda_currents, gaba_currents, gaba_conductances, dt


In [ ]:
soma_v_master = []
dend_v_master = []
if impedance:
    soma_z_master = []
    dend_z_master = []

if 'timing_range' in globals() and variable_detector(timing_range):
    variable ='timing'
elif 'gaba_range' in globals() and variable_detector(gbar_gaba_range):
    variable ='gGABA'
elif 'num_gabas_range' in globals():
    variable ='nGABA'
    nGABAs = 0
elif 'holding_current_range' in globals() and variable_detector(holding_current_range):
    variable ='hc'

data_dict = {
    'soma_v_data': [],
    'dend_v_data': [],
    'soma_z_data': [],
    'dend_z_data': [],
    'v_data': [],
    'vdend_data': [],
    'Ca_data': [],
    'Cadend_data': [],
    'imp_dend_data': [],
    'imp_soma_data': [],
    't_data': [],
    'gt': [],
    'v_tree': {},
    'Ca_tree': {},
    'i_mechs': {},
    'i_ampa': {},
    'i_nmda': {},
    'i_gaba': {},
    'g_gaba': {},
    'dend_gaba': [],
    'gaba_locations': [],
     variable: []
}   


for ii in tqdm(range(0, len(gaba_range), 1)): # gaba burst every 10 ms tqdm(range(140, 240, 10))
    timing = timing_range[ii]
    gaba = gaba_range[ii]
    glut = glut_range[ii]
    num_gabas = num_gabas_range[ii]
    num_gluts = num_gluts_range[ii]

    if variable == 'nGABA':
        num_gabas = num_gabas_range[ii]
        if ii % 2 == 0: 
            nGABAs = nGABAs + num_gabas
        ind2 = len(dend_gaba_list)
        ind1 = len(dend_gaba_list) - nGABAs
        dend_gaba = dend_gaba_list[ind1:ind2]

    if vary_hc:
        holding_current = holding_current_range[ii]
        print('holding current ', holding_current)
    if vary_gaba:
        glut_time = stim_time
        gaba_time = timing
    else:
        glut_time = timing
        gaba_time = stim_time
    if tonic:
        gbar_gaba = gbar_gaba_range[ii]
        print('tonic GABA ', gbar_gaba)

    # Run sim
    plot = True
    if impedance:
        if return_Ca:
            v_soma, v_dend, v_all, Ca_soma, Ca_dend, Ca_all, t, i_mechs_dend, ampa_currents, nmda_currents, gaba_currents, gaba_conductances, zvec1, zvec2, dt = phasic_current_clamp(
                sim_time, glut_time, gaba_time, glut, num_gluts, dend_glut, gaba, num_gabas,
                gaba_weight, dend_gaba, gaba_locations, spine_neck_diam, return_currents, cell_type=cell_type, plot=plot)
        else:
            v_soma, v_dend, v_all, t, i_mechs_dend, ampa_currents, nmda_currents, gaba_currents, gaba_conductances, zvec1, zvec2, dt = phasic_current_clamp(
                sim_time, glut_time, gaba_time, glut, num_gluts, dend_glut, gaba, num_gabas,
                gaba_weight, dend_gaba, gaba_locations, spine_neck_diam, return_currents, cell_type=cell_type, plot=plot)
    else:
        if return_Ca:
            v_soma, v_dend, v_all, Ca_soma, Ca_dend, Ca_all, t, i_mechs_dend, ampa_currents, nmda_currents, gaba_currents, gaba_conductances, dt = phasic_current_clamp(
                sim_time, glut_time, gaba_time, glut, num_gluts, dend_glut, gaba, num_gabas,
                gaba_weight, dend_gaba, gaba_locations, spine_neck_diam, return_currents, cell_type=cell_type, plot=plot)
        else:
            v_soma, v_dend, v_all, t, i_mechs_dend, ampa_currents, nmda_currents, gaba_currents, gaba_conductances, dt = phasic_current_clamp(
                sim_time, glut_time, gaba_time, glut, num_gluts, dend_glut, gaba, num_gabas,
                gaba_weight, dend_gaba, gaba_locations, spine_neck_diam, return_currents, cell_type=cell_type, plot=plot)


    if variable == 'timing':
        protocol = "{}:{}".format(variable, timing)
        data_dict[variable].append(timing)
    if variable == 'nGABA':
        protocol = "{}:{} GLUT:{}".format(variable, nGABAs, glut)
        data_dict[variable].append(num_gabas)
    if variable == 'gGABA':
        protocol = "{}:{}".format(variable, gbar_gaba)
        data_dict[variable].append(gbar_gaba)
    if variable == 'hc':
        protocol = "{}:{}".format(variable, hc)
        data_dict[variable].append(hc)

    data_dict['v_tree'][protocol] = pd.DataFrame(v_all)
    if return_Ca:
        data_dict['Ca_tree'][protocol] = pd.DataFrame(Ca_all)

    ind1 = 1
    ind2 = int(sim_time / dt)
    t2 = np.arange(1, int(sim_time / dt), 1) * dt - dt
    data_dict['soma_v_data'].append(go.Scatter(x=t2, y=extract2(v_soma)[ind1:ind2], name='timing:{}'.format(timing)))
    data_dict['dend_v_data'].append(go.Scatter(x=t2, y=extract2(v_dend)[ind1:ind2], name='timing:{}'.format(timing)))
    soma_v_master.append(go.Scatter(x=t2, y=extract2(v_soma)[ind1:ind2], name='timing:{}'.format(timing)))
    dend_v_master.append(go.Scatter(x=t2, y=extract2(v_dend)[ind1:ind2], name='timing:{}'.format(timing)))
    if impedance:
        data_dict['soma_z_data'].append(go.Scatter(x=t2, y=extract2(zvec2)[ind1:ind2], name='timing:{}'.format(timing)))
        data_dict['dend_z_data'].append(go.Scatter(x=t2, y=extract2(zvec1)[ind1:ind2], name='timing:{}'.format(timing)))
        soma_z_master.append(go.Scatter(x=t2, y=extract2(zvec2)[ind1:ind2], name='timing:{}'.format(timing)))
        dend_z_master.append(go.Scatter(x=t2, y=extract2(zvec1)[ind1:ind2], name='timing:{}'.format(timing)))

    data_dict['v_data'].append(np.asarray(v_soma))
    if return_Ca:
        data_dict['Ca_data'].append(np.asarray(Ca_soma))
    data_dict['t_data'].append(np.asarray(t))
    data_dict['gt'].append([timing] * len(np.asarray(t)))
    data_dict['vdend_data'].append(np.asarray(v_dend))
    if return_Ca:
        data_dict['Cadend_data'].append(np.asarray(Ca_dend))
    if impedance:
        data_dict['imp_dend_data'].append(np.asarray(zvec1))
        data_dict['imp_soma_data'].append(np.asarray(zvec2))

    data_dict['dend_gaba'].append(np.asarray(dend_gaba))
    data_dict['gaba_locations'].append(np.asarray(gaba_locations))

    if return_currents:
        i_mechs_dend_df = list2df(i_mechs_dend)
        if vary_gaba:
            data_dict['i_mechs'][protocol] = i_mechs_dend_df
            data_dict['i_ampa'][protocol] = pd.DataFrame(ampa_currents).transpose()
            data_dict['i_nmda'][protocol] = pd.DataFrame(nmda_currents).transpose()
            data_dict['i_gaba'][protocol] = pd.DataFrame(gaba_currents).transpose()
            data_dict['g_gaba'][protocol] = pd.DataFrame(gaba_conductances).transpose()
        else:
            data_dict['i_mechs'][protocol] = i_mechs_dend_df
            data_dict['i_ampa'][protocol] = pd.DataFrame(ampa_currents).transpose()
            data_dict['i_nmda'][protocol] = pd.DataFrame(nmda_currents).transpose()
            data_dict['i_gaba'][protocol] = pd.DataFrame(gaba_currents).transpose()
            data_dict['g_gaba'][protocol] = pd.DataFrame(gaba_conductances).transpose()

if save:
    save_all()

if model == 2 or cell_type == 'ispn':
    yrange_soma1=[-85,-50]
    yrange_dend1=[-85,-20]
else:
    yrange_soma1=[-85,-60]
    yrange_dend1=[-85,-30]  

if sim == 9 or sim == 10:
    fig_soma_master, fig_dend_master =  plot3(somaV=soma_v_master, dendV=dend_v_master, yaxis='V (mV)', yrange_soma=yrange_soma1, yrange_dend=yrange_soma1, stim_time=stim_time, sim_time=sim_time, black_trace=black_trace, gray_trace=gray_trace, err_bar=50, baseline=20, dt=dt)    
else:
    fig_soma_master, fig_dend_master =  plot3(somaV=soma_v_master, dendV=dend_v_master, yaxis='V (mV)', yrange_soma=yrange_soma1, yrange_dend=yrange_dend1, stim_time=stim_time, sim_time=sim_time, black_trace=black_trace, gray_trace=gray_trace, err_bar=50, baseline=20, dt=dt)    

fig_dend_master.show()
fig_soma_master.show()
if save: save_fig2(soma_fig=fig_soma_master, dend_fig=fig_dend_master, cell_type=cell_type, model=model, physiological=physiological, sim=sim)    
if impedance:
    if sim == 9 or sim == 10:
        fig_z_soma_master, fig_z_dend_master =  plot3(somaV=soma_z_master, dendV=dend_z_master, yaxis='MOhm', yrange_soma=[0,80], yrange_dend=[50,300], stim_time=stim_time, sim_time=sim_time, black_trace=black_trace, gray_trace=gray_trace, err_bar=50, baseline=20, dt=dt)    
    else:
        fig_z_soma_master, fig_z_dend_master =  plot3(somaV=soma_z_master, dendV=dend_z_master, yaxis='MOhm', yrange_soma=[0,100], yrange_dend=[0,800], stim_time=stim_time, sim_time=sim_time, black_trace=black_trace, gray_trace=gray_trace, err_bar=50, baseline=20, dt=dt)    
    fig_z_dend_master.show()
    fig_z_soma_master.show()
    if save: save_fig2(soma_fig=fig_z_soma_master, dend_fig=fig_z_dend_master, cell_type=cell_type, model=model, physiological=physiological, sim=sim)   
